In [4]:
using DrWatson
using Pkg
@quickactivate "toysector"
using CSV, DelimitedFiles, HypothesisTests, BSON, Distributions, Random 
using SpinModels, Optim, Parameters, CUDA, LinearAlgebra, Statistics, Printf, Flux
using BenchmarkTools, PyPlot
import StatsBase.countmap
import DrWatson.savename

include( srcdir( "fitting.jl"   ) )
include( srcdir( "toysector.jl" ) )
include( srcdir( "false_positive_funcs.jl" ) )
include( srcdir( "samplers.jl"             ) )

drawsamples!! (generic function with 3 methods)

# helper funcs

In [3]:
function get_froJ(J::AbstractArray{T,4} ,p::Real=2) where T<:Number
    N = size(J,2)
    froJ= [ norm(J[:,i,:,j], p) for i ∈ 1:N, j ∈ 1:N ] 
end

function get_froh(h::AbstractMatrix{T} ,p::Real=2) where T<:Number
    N = size( h,2 )
    froh = [ norm(h[:,i], p) for i ∈ 1:N ]
end

function get_froW(W::AbstractArray{T,3}, p::Real=2) where T<:Number
    P, _, N = size(W)
    froW = [ norm(W[μ,:,i],p) for μ ∈ 1:P, i ∈ 1:N  ]
end

function get_fro(A::AbstractArray, p::Real=2)
    ndims(A) == 2 && return get_froh(A,p)
    ndims(A) == 3 && return get_froW(A,p)
    ndims(A) == 4 && return get_froJ(A,p) 
end

im20(mat)= im20( mat,  begin _, ax = subplots(); ax end )
function im20(mat, ax)
    im = ax.imshow(mat )

    dimy, dimx = size(mat)
    
    stepx = round(dimx/10) == 0 ? 1 : Int(round(dimx/10))
    stepy = round(dimy/10) == 0 ? 1 : Int(round(dimy/10))
    
    # Major ticks
    ax.set_xticks(0:stepx:dimx-1)
    ax.set_yticks(0:stepy:dimy-1)

    # Labels for major ticks
    ax.set_xticklabels(1:stepx:dimx)
    ax.set_yticklabels(1:stepy:dimy)

    # Minor ticks
    ax.set_xticks(-.5:1:dimx, minor=true)
    ax.set_yticks(-.5:1:dimy, minor=true)

    # # Gridlines based on minor ticks
    # minorticks_off()
    (dimx < 50) && (dimy < 50) ? ax.grid(true, which="minor", color="k", linestyle="-", linewidth=1.5) : nothing
#     colorbar()\
    sca(ax)
    tick_params(bottom = false, left = false, which = "both")
    
    return im
end

im35(mat)= im35( mat,  begin _, ax = subplots(); ax end,  )
function im35(mat, ax, cmap="cividis", hideticklabels=false)
    im = ax.imshow(mat, cmap=cmap )

    dimy, dimx = size(mat) 
    stepx = round(dimx/10) == 0 ? 1 : Int(round(dimx/10))
    stepy = round(dimy/10) == 0 ? 1 : Int(round(dimy/10))

    ax.set_xticks([0,9,28,34])
    ax.set_yticks([0,9,28,34])
    ax.set_xticklabels( [1,10,29,35],  fontsize=16 )
    ax.set_yticklabels( [1,10,29,35],  fontsize=16 )
    
    ax.tick_params(bottom = false, left = false, which = "both")
    
    ax.set_xticks(-.5:1:dimx, minor=true)
    ax.set_yticks(-.5:1:dimy, minor=true)
    (dimx < 50) && (dimy < 50) ? ax.grid(true, which="minor", 
        color="k", linestyle="-", linewidth=0.5) : nothing
    
    
    if hideticklabels == false
        ax.xaxis.set_tick_params(labeltop=true)
        ax.xaxis.set_tick_params(labelbottom=false)
    else
        ax.yaxis.set_tick_params(labelleft=false)
        ax.xaxis.set_tick_params(labelbottom=false)
    end
    
   
    
    # Major ticks
    # ax.set_xticks(0:stepx:dimx-1)
    # ax.set_yticks(0:stepy:dimy-1)
    # # Labels for major ticks
    # ax.set_xticklabels(1:stepx:dimx)
    # ax.set_yticklabels(1:stepy:dimy)  
    
    ax.vlines([10,28].-0.5, -0.5, dimy-0.5, colors="white", linewidth=2)
    ax.hlines([10,28].-0.5, -0.5, dimx-0.5, colors="white", linewidth=2)
    return im
end


im35 (generic function with 4 methods)

# specify models to fit

In [28]:
folds = 5
seed = 123
Random.seed!(seed)
calc_f_pos = false
nsamples_for_fpos = 10_000;
calc_H = false

false

In [38]:
samples_to_fit_all = Dict{Symbol, Any}(
    :M => [500], #nsamples
#     :T_sec => [0.2, 0.25, 0.3, 0.4, 0.5, 0.6]  
    :T_sec => [1.,0.4, 0.2],
    :m => [1,2,3]
    )

modls = [:Pairwise]# , :RBM]#, :SRBM]

params_sets = Dict{Symbol,Any}(
        :Pairwise => Dict{Symbol, Any}(
        :reg_J => [(10f0^log10λ, 2) for log10λ ∈ -8:1],
        :reg_h => [(10f0^log10λ, 2) for log10λ ∈ 2:2],
        :epochs => 20_000,
        :showevery => 500,
        :progTol => 10^-6
    )
     # ,
     #     :RBM => Dict{Symbol, Any}(
     #     :reg_W => [(10f0^log10λ, 2) for log10λ ∈ -8:-1],
     #     :reg_h => [(10f0^log10λ, 2) for log10λ ∈ -4:2],
     #     :P => [10,20,30,40,50],
     #     :epochs => 20_000,
     #     :showevery => 500,
     #     :progTol => 10^-6
     # )
#     ,
#     :SRBM => Dict{Symbol, Any}(
#         :reg_J => [(10f0^log10λ, 2) for log10λ ∈ -6:-1],
#         :reg_W => [(10f0^log10λ, 2) for log10λ ∈ -8:-1],
#         :reg_h => [(10f0^log10λ, 2) for log10λ ∈ -4:2],
#         :P => collect(10:20:50),
#         :epochs => 20_000,
#         :showevery => 50,
#         :progTol => 10^-6
#     )
)

samples_to_fit_dict_list = dict_list(samples_to_fit_all)

9-element Vector{Dict{Symbol, Real}}:
 Dict(:m => 1, :M => 500, :T_sec => 1.0)
 Dict(:m => 2, :M => 500, :T_sec => 1.0)
 Dict(:m => 3, :M => 500, :T_sec => 1.0)
 Dict(:m => 1, :M => 500, :T_sec => 0.4)
 Dict(:m => 2, :M => 500, :T_sec => 0.4)
 Dict(:m => 3, :M => 500, :T_sec => 0.4)
 Dict(:m => 1, :M => 500, :T_sec => 0.2)
 Dict(:m => 2, :M => 500, :T_sec => 0.2)
 Dict(:m => 3, :M => 500, :T_sec => 0.2)

# fit the models for each dataset

In [41]:
for k in 1:length(samples_to_fit_dict_list)


samples_to_fit_info=samples_to_fit_dict_list[k]
sampledir(args...) = datadir("toysector_q=5", 
    "nsamples=$(samples_to_fit_info[:M])", args...)

z = BSON.load(
    sampledir( @sprintf("sector_temp=%.2f_m=%i_samples.bson", 
                samples_to_fit_info[:T_sec], samples_to_fit_info[:m]) ) 
    )[:samples];
z = Float32.(z)
q,N,_=size(z)

### make into cuda array if possible
z = CUDA.functional() ? cu(z) : z
@printf "%s\n" "Array type: $(typeof(z))"
𝕏 = kfold( Float32.(z), folds )

fitsdir(args...) = sampledir("fitted_models", 
    @sprintf("sector_temp=%.2f_m=%i", 
        samples_to_fit_info[:T_sec], samples_to_fit_info[:m]), 
    args...)

sweep_dicts = Dict{Symbol, Any}()
for mdl in modls
    sweep_dicts[mdl] = dict_list( params_sets[mdl] )
end
for mdl in modls
    mdldir(args...) = fitsdir(string(mdl), args...)
    isdir(mdldir()) || mkpath(mdldir())
    
    println(mdldir())
    swp_ds_string = mdldir( string(mdl)*"_all_results.bson")
    
    ### do hyperparam sweep
    swp_ds = sweep_dicts[mdl] #alias
    @printf "\n"
    @printf "Begin sweep of %s model\n" string(mdl)
    for d in sweep_dicts[mdl]
        @printf "\n"
        @printf "-----------------------------------------------\n"
        @printf "%s\n" savename(d)
	    if isfile(mdldir(savename(d)*".bson"))
            merge!(d, BSON.load(mdldir(savename(d)*".bson"))[:d])
            @printf "model alrealy fit. skipping\n" 
            continue
        end
        
        @printf "-----------------------------------------------\n"
        ### 5 fold fit
        θ = mdl == :Pairwise ? Pairwise(; q=q , N=N, similarto=z) : eval(mdl)(q=q,N=N,P=d[:P], similarto=z)
        SpinModels.random!(θ)
        _, 𝕋, 𝕍 = kfold_sgdfit!(θ, 𝕏; d... )
        ### fit all data to model
        SpinModels.random!(θ)
        @printf "train on whole dataset\n"
        intrace = sgdfit!(θ, z; d... )
        inloss = intrace[:f_training][end] 
        
        ###calculate false_pos_rate
        false_pos_rate_entropy = ( calc_f_pos || (calc_H && (N*log(q)) < 21. ) ) ? get_f_pos_rate( θsec, θ, 
            sec_energy_buffer, all_states_buffer, nsamples_for_fpos ) : (nothing, nothing)
        false_pos_rate = false_pos_rate_entropy[1]
        fit_entropy = false_pos_rate_entropy[2]
        
        ### put results in dictionary
        results = Dict{Symbol, Any}( :𝕍 => 𝕍, :𝕋 => 𝕋, :intrace => intrace, :inloss => inloss, 
             :mean𝕍 => mean(𝕍), :std𝕍 => std(𝕍), :false_pos_rate => false_pos_rate, 
                :fit_entropy => fit_entropy, :true_entropy => nothing)  
        merge!(d, results)
        
        
        ### save results
        BSON.@save mdldir(savename(d)*".bson") d
        vecθ = collect(veccopy(θ))
        BSON.@save mdldir(savename(d)*"_model.bson") vecθ
    end
#     #also save the vector of dictionaries with results
    BSON.@save swp_ds_string swp_ds
end
    
    
end

Array type: Array{Float32, 3}
/Users/pfields/Git/toysector/data/toysector_q=5/nsamples=500/fitted_models/sector_temp=1.00_m=1/Pairwise

Begin sweep of Pairwise model

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-8.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-7.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-6.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-5.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-4.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-3.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-2.0
model alrealy fit. skipping

-----------------------------------------------
L2_regh=1e2.0_regJ=1e-1.0
model alrealy fit. skipping

---------

# find best model for each dataset

In [61]:
for k in 1:length(samples_to_fit_dict_list)

    samples_to_fit_info=samples_to_fit_dict_list[k]
    sampledir(args...) = datadir("toysector_q=5", 
        "nsamples=$(samples_to_fit_info[:M])", args...)


    fitsdir(args...) = sampledir("fitted_models", 
        @sprintf("sector_temp=%.2f_m=%i", 
            samples_to_fit_info[:T_sec], samples_to_fit_info[:m]), 
        args...)
    
    sweep_dicts = Dict{Symbol, Any}()

    for mdl in modls
        mdldir(args...) = fitsdir(string(mdl), args...)

        println(mdldir())
        swp_ds_string = mdldir( string(mdl)*"_all_results.bson")

        ### do hyperparam sweep
        # swp_ds = sweep_dicts[mdl] #alias

        BSON.@load swp_ds_string swp_ds

        # @show keys(swp_ds[1])
        
        sweep_dicts[mdl]=swp_ds
    end
    
    losses_pw   = pull_from_sweep(sweep_dicts[:Pairwise], 
    [:reg_h, :reg_J, :mean𝕍] 
    )
    best_idx_pw = findmin(losses_pw[:,end])[2]
    best_pw_reg_h, best_pw_reg_J, _ = losses_pw[best_idx_pw, :]

    ### put that model in accessible part of folder
    best_model_dict_pw=filter( d -> ( (d[:reg_J] == best_pw_reg_J) && (d[:reg_h] == best_pw_reg_h) ) , sweep_dicts[:Pairwise] )[1]
    cp( fitsdir("Pairwise", savename(best_model_dict_pw)*"_model.bson") , fitsdir(savename(best_model_dict_pw)*"_bestmodel.bson") ; force=true) 
    cp( fitsdir("Pairwise", savename(best_model_dict_pw)*".bson") , fitsdir(savename(best_model_dict_pw)*"_best.bson") ; force = true) 
end

/Users/pfields/Git/toysector/data/toysector_q=5/nsamples=500/fitted_models/sector_temp=1.00_m=1/Pairwise
keys(swp_ds[1]) = [:progTol, :true_entropy, :false_pos_rate, :reg_J, :fit_entropy, :reg_h, :intrace, :𝕍, :std𝕍, :mean𝕍, :showevery, :epochs, :𝕋, :inloss]
/Users/pfields/Git/toysector/data/toysector_q=5/nsamples=500/fitted_models/sector_temp=1.00_m=2/Pairwise
keys(swp_ds[1]) = [:progTol, :true_entropy, :false_pos_rate, :reg_J, :fit_entropy, :reg_h, :intrace, :𝕍, :std𝕍, :mean𝕍, :showevery, :epochs, :𝕋, :inloss]
/Users/pfields/Git/toysector/data/toysector_q=5/nsamples=500/fitted_models/sector_temp=1.00_m=3/Pairwise
keys(swp_ds[1]) = [:progTol, :true_entropy, :false_pos_rate, :reg_J, :fit_entropy, :reg_h, :intrace, :𝕍, :std𝕍, :mean𝕍, :showevery, :epochs, :𝕋, :inloss]
/Users/pfields/Git/toysector/data/toysector_q=5/nsamples=500/fitted_models/sector_temp=0.40_m=1/Pairwise
keys(swp_ds[1]) = [:progTol, :true_entropy, :false_pos_rate, :reg_J, :fit_entropy, :reg_h, :intrace, :𝕍, :std𝕍, :mean𝕍